<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH16_03_%EC%85%80%ED%94%84%EC%96%B4%ED%85%90%EC%85%98%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98(Multi_head_Self_Attention_for_Text_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

트랜스포머는 RNN 계열의 seq2seq를 대체하기 위해 등장함. 그리고 트랜스포머의 인코더는 RNN 인코더를, 트랜스포머의 디코더는 RNN 디코더를 대체할 수 있었음

트랜스포머의 인코더는 셀프 어텐션이라는 메커니즘을 통해 문장을 이해함. RNN과 그 동작방식은 다르지만, RNN이 텍스트 분류나 개체명인식과 같은 다양한 자연어 처리태스크에 쓰일 수 있다면, 트랜스포머의 인코더 또한 가능할 것임


실제, 트랜스포머의 인코더는 다양한 분야의 자연어 처리 태스크에서 사용될 수 있었고, 이 아이디어는 후에 배우게 될 BERT라는 모델로 이어지게 됨

이번 챕터에서 트랜스포머의 인코더를 사용하여 텍스트 분류를 수행함

### 1. 멀티 헤드 어텐션
--------------------
우선 트랜스포머의 인코더 첫번째 서브층인 멀티헤드 어텐션층을 클래스로 구현함

In [1]:
import tensorflow as tf

In [2]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads=8):
    super(MultiHeadAttention, self).__init__()
    self.embedding_dim = embedding_dim  # d_model
    self.num_heads = num_heads

    assert embedding_dim % self.num_heads == 0

    self.projection_dim = embedding_dim // num_heads 
    self.query_dense = tf.keras.layers.Dense(embedding_dim)
    self.key_dense = tf.keras.layers.Dense(embedding_dim)
    self.value_dense = tf.keras.layers.Dense(embedding_dim)
    self.dense = tf.keras.layers.Dense(embedding_dim)

  def scaled_dot_product_attention(self, query, key, value):
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    attention_weights = tf.nn.softmax(logits, axis=-1)  # 소프트맥스 함수는 마지막 차원인 Key의 문장길이 방향으로 수행
    output = tf.matmul(attention_weights, value)
    return output, attention_weights

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, inputs):
    #inputs.shape = [batch_size, seq_len, embedding_dim]
    batch_size = tf.shape(inputs)[0]

    # (batch_size, seq_len, embedding_dim)
    query = self.query_dense(inputs)
    key = self.key_dense(inputs)
    value = self.value_dense(inputs)

    # (batch_size, num_heads, seq_len, projection_dim)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # (batch_size, seq_len, embedding_dim)
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
    outputs = self.dense(concat_attention)
    return outputs



### 2. 인코더 설계하기
--------------------
멀티헤드 어텐션에 두번째 서브층인 포지션 와이즈 FFNN을 추가하여 인코더 클래스를 설계함

In [3]:
class TransformerBlock(tf.keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
    super(TransformerBlock, self).__init__()
    self.att = MultiHeadAttention(embedding_dim, num_heads)
    self.ffn = tf.keras.Sequential(
        [tf.keras.layers.Dense(dff, activation='relu'), tf.keras.layers.Dense(embedding_dim)])
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, inputs, training):
    attn_output = self.att(inputs)  # 첫번째 서브층 : 멀티헤드어텐션
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(inputs + attn_output)  # Add & Norm
    ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 FFNN
    ffn_output = self.dropout2(ffn_output, training=training)
    return self.layernorm2(out1 + ffn_output)  # Add & Norm

### 3. 포지션 임베딩
--------------
앞서 트랜스포머를 설명할 때는 포지셔널 인코딩을 사용했지만, 여기에서는 위치정보자체를 학습하기도록 하는 **포지션 임베딩 방법**을 사용함

이는 뒤에서 배우게 될 BERT에서 사용하는 방법으로, 포지션 임베딩은 임베딩층을 사용하되, 위치벡터를 학습하도록 하므로 임베딩 층의 첫번째 인자로 단어집합 크기가 아니라, 문장의 최대 길이를 넣어줌


In [4]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
  def __init__(self, max_len, vocab_size, embedding_dim):
    super(TokenAndPositionEmbedding, self).__init__()
    self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

  def call(self, x):
    max_len = tf.shape(x)[-1]
    positions = tf.range(start=0, limit=max_len, delta=1)
    positions = self.pos_emb(positions)
    x = self.token_emb(x)
    return x + positions

### 4. 데이터 로드 및 전처리
-------------

In [5]:
vocab_size = 20000  # 빈도수 상위 2만개의 단어만 사용
max_len = 200  # 문장의 최대 길이

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))

훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [6]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### 5. 트랜스포머를 이용한 IMDB 리뷰 분류
--------------------


In [7]:
embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
num_heads = 2 # 어텐션 헤드 수
dff = 32  # 포지션 와이즈 FFNN 은닉층의 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [10]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
782/782 [==============================] - 56s 69ms/step - loss: 0.1372 - accuracy: 0.9506 - val_loss: 0.5929 - val_accuracy: 0.8048
Epoch 2/3
782/782 [==============================] - 61s 78ms/step - loss: 0.0929 - accuracy: 0.9692 - val_loss: 0.4894 - val_accuracy: 0.8521
Epoch 3/3
782/782 [==============================] - 65s 84ms/step - loss: 0.0605 - accuracy: 0.9800 - val_loss: 0.5960 - val_accuracy: 0.8442


In [9]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 14s 17ms/step - loss: 0.3375 - accuracy: 0.8628
테스트 정확도: 0.8628


In [11]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 13s 17ms/step - loss: 0.5960 - accuracy: 0.8442
테스트 정확도: 0.8442
